# Analysis

The first thing to do is to get the dataset with all the samples:
- it contains the main usage infos
- it contains the __machine_spec__ data
- it contains the __machine_metric__ data (check better what it means)


## Some libraries

In [1]:
import sys
import logging

## LOGGING
nblog = open("alibaba_data-evaluation.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

In [3]:
import dask.array as da
import dask.distributed


In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
from scipy.stats import norm
import numpy as np

import matplotlib as mpl
from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [6]:
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax

In [9]:
import hdbscan
from sklearn.cluster import OPTICS

In [10]:
from collections import defaultdict
from collections import Counter

In [11]:
import random

In [12]:
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

In [13]:
import shap

In [14]:
from xgboost import plot_importance

from xgboost import to_graphviz

from sklearn.metrics import classification_report,confusion_matrix

In [15]:
import pickle

## Load the dataset

In [16]:
sampled_workload_data = pd.read_csv('experiments/100_001_sampled_workload_data_full.csv', index_col=0)

In [17]:
sampled_workload_data

,inst_id,user,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type,group,workload,inst_name,worker_name,machine_x,gpu_name,cpu_usage,gpu_wrk_util,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,read,write,read_count,write_count,duration,gpu_type_machine_spec,cap_cpu_machine_spec,cap_mem_machine_spec,cap_gpu_machine_spec,machine_y,machine_cpu_iowait_machine_metric,machine_cpu_kernel_machine_metric,machine_cpu_usr_machine_metric,machine_gpu_machine_metric,machine_load_1_machine_metric,machine_net_receive_machine_metric,machine_num_worker_machine_metric,machine_cpu_machine_metric
job_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71712e3c961b5bbdb2ce0440,c5be8813bf90aaf9ca6f02f744ccd95596e9850928a498...,2cbd33b08024,ps,5.0,Terminated,4625696.0,4628399.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,c2e0be7577f427c47d77dd388f5bad181447f0bd21bc9e...,04e8a1d93c8a93a3a0a10466cd2f25132dd14c73293d24...,137e86ce1bc508e64d5fc5ee,/dev/nvidia1,34.465789,7.541667,1.204708,2.723633,0.347158,0.829102,2.068066e+06,3.426616e+05,312.218182,219.376623,2703.0,P100,64.0,512.0,2.0,137e86ce1bc508e64d5fc5ee,0.005480,2.603109,9.889830,45.808696,7.668093,2.620698e+08,5.335775,12.498419
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,1e999417385ceac6fc6908fc833dbc1bf7ccb5ab9e1af7...,3c41010cedf0cc85aa05e197131d20a5684be2c7dd2844...,e43590f20ce91e21cf8f0062,/dev/nvidia0,52.794326,12.049123,1.505760,2.671875,0.445045,0.827148,1.234028e+06,3.724553e+05,264.849650,173.374126,1595.0,P100,64.0,512.0,2.0,e43590f20ce91e21cf8f0062,0.001801,10.529087,28.466644,80.292014,34.026547,1.068209e+08,3.888263,38.997531
b68f25260271d65caccca8e7,5229275f7f92865554f934e97f576ef99db0c079bba59d...,d4d51aca8806,worker,16.0,Terminated,4058070.0,4058321.0,50.0,29.296875,100.0,T4,8bc721404a5d34c9f68829312d324994,bert,f004b4f39329b0c12a6b8de546f95fe477885ed004c856...,db23db95b2af54b7a37b4cc1beb4935c57a5c385d654a8...,f1f4f45a9474a2fd362d76fe,/dev/nvidia0,75.052632,23.727273,1.037900,1.853516,0.445890,0.774414,1.010761e+07,3.781242e+06,2401.913043,590.652174,251.0,T4,96.0,512.0,2.0,f1f4f45a9474a2fd362d76fe,0.002509,2.282917,1.616539,28.132630,4.036872,1.746246e+07,1.537805,3.901965
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,570f01d774b50c6d0026967d83b1c3403d093ffbe4c474...,3c8e0b3564bf056fbd705402c5d3429e8a89643d4c8d6c...,3c4af9c9b5db2d0a07860338,/dev/nvidia1,92.604240,11.706294,1.481600,2.667969,0.449140,0.829102,2.358233e+06,4.527896e+05,404.843206,271.797909,1595.0,P100,64.0,512.0,2.0,3c4af9c9b5db2d0a07860338,0.002109,1.872507,6.680456,20.784838,5.452154,1.350479e+08,5.147730,8.555072
446cd4921e0f00ab5ce1c2ea,12b9ec70cc7c63ebc7184b8a0c363513a8c3cb19746c74...,2cbd33b08024,ps,2.0,Terminated,5521030.0,5526746.0,600.0,20.000000,NaN,NaN,4ac7f9c48a6913dfa81169f7d6a150fa,bert,a5d7f3cf14cc3baef1b59eab27895e2222f2302735d873...,c34f6c58302ec92032293a469d2cbfaccba65ff97203b6...,d4f54f7e5cf18341942c47ef,/dev/nvidia0,110.590788,13.005300,2.357810,3.056641,0.645605,0.827148,1.037695e+06,1.527555e+05,201.449250,181.084731,5716.0,P100,64.0,512.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e49cae3b33b5c205ed8542b0,5283a27d4f19d767256f97c18880a70d69f4d97ca655a3...,47baba33cffe,ps,1.0,Terminated,5890601.0,5893675.0,600.0,29.296875,NaN,NaN,7389796d26bc0d9c8bae9700a46e88ca,xlnet,8d6b9f3b9ef8539217bc16388fb88e18542c0048677344...,20fb53a63dab2ac5da0e2bc08328ee3835d5b0bf0a78f2...,2988f6d00a08192b50b767f4,/dev/nvidia1,127.340568,81.609635,1.940527,1.995117,1.200031,1.213867,1.941028e+06,1.165604e+06,402.442786,171.301824,

## Load hdbscan results

In [18]:
hdbscan_clusterer = pickle.load(open("experiments/hdbscan_300_power_transform_euclidean.pkl", "rb"))

In [19]:
label_stats = Counter(hdbscan_clusterer.labels_)

label_stats.most_common(10)


[(-1, 24603),
 (8, 12522),
 (9, 11612),
 (25, 7754),
 (23, 7474),
 (1, 5751),
 (12, 5501),
 (3, 3043),
 (13, 2287),
 (20, 2128)]

In [20]:
sampled_workload_data_with_profiles = sampled_workload_data.copy()
sampled_workload_data_with_profiles['profile'] = hdbscan_clusterer.labels_
sampled_workload_data_with_profiles

,inst_id,user,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type,group,workload,inst_name,worker_name,machine_x,gpu_name,cpu_usage,gpu_wrk_util,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,read,write,read_count,write_count,duration,gpu_type_machine_spec,cap_cpu_machine_spec,cap_mem_machine_spec,cap_gpu_machine_spec,machine_y,machine_cpu_iowait_machine_metric,machine_cpu_kernel_machine_metric,machine_cpu_usr_machine_metric,machine_gpu_machine_metric,machine_load_1_machine_metric,machine_net_receive_machine_metric,machine_num_worker_machine_metric,machine_cpu_machine_metric,profile
job_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71712e3c961b5bbdb2ce0440,c5be8813bf90aaf9ca6f02f744ccd95596e9850928a498...,2cbd33b08024,ps,5.0,Terminated,4625696.0,4628399.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,c2e0be7577f427c47d77dd388f5bad181447f0bd21bc9e...,04e8a1d93c8a93a3a0a10466cd2f25132dd14c73293d24...,137e86ce1bc508e64d5fc5ee,/dev/nvidia1,34.465789,7.541667,1.204708,2.723633,0.347158,0.829102,2.068066e+06,3.426616e+05,312.218182,219.376623,2703.0,P100,64.0,512.0,2.0,137e86ce1bc508e64d5fc5ee,0.005480,2.603109,9.889830,45.808696,7.668093,2.620698e+08,5.335775,12.498419,9
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,1e999417385ceac6fc6908fc833dbc1bf7ccb5ab9e1af7...,3c41010cedf0cc85aa05e197131d20a5684be2c7dd2844...,e43590f20ce91e21cf8f0062,/dev/nvidia0,52.794326,12.049123,1.505760,2.671875,0.445045,0.827148,1.234028e+06,3.724553e+05,264.849650,173.374126,1595.0,P100,64.0,512.0,2.0,e43590f20ce91e21cf8f0062,0.001801,10.529087,28.466644,80.292014,34.026547,1.068209e+08,3.888263,38.997531,9
b68f25260271d65caccca8e7,5229275f7f92865554f934e97f576ef99db0c079bba59d...,d4d51aca8806,worker,16.0,Terminated,4058070.0,4058321.0,50.0,29.296875,100.0,T4,8bc721404a5d34c9f68829312d324994,bert,f004b4f39329b0c12a6b8de546f95fe477885ed004c856...,db23db95b2af54b7a37b4cc1beb4935c57a5c385d654a8...,f1f4f45a9474a2fd362d76fe,/dev/nvidia0,75.052632,23.727273,1.037900,1.853516,0.445890,0.774414,1.010761e+07,3.781242e+06,2401.913043,590.652174,251.0,T4,96.0,512.0,2.0,f1f4f45a9474a2fd362d76fe,0.002509,2.282917,1.616539,28.132630,4.036872,1.746246e+07,1.537805,3.901965,0
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,570f01d774b50c6d0026967d83b1c3403d093ffbe4c474...,3c8e0b3564bf056fbd705402c5d3429e8a89643d4c8d6c...,3c4af9c9b5db2d0a07860338,/dev/nvidia1,92.604240,11.706294,1.481600,2.667969,0.449140,0.829102,2.358233e+06,4.527896e+05,404.843206,271.797909,1595.0,P100,64.0,512.0,2.0,3c4af9c9b5db2d0a07860338,0.002109,1.872507,6.680456,20.784838,5.452154,1.350479e+08,5.147730,8.555072,9
446cd4921e0f00ab5ce1c2ea,12b9ec70cc7c63ebc7184b8a0c363513a8c3cb19746c74...,2cbd33b08024,ps,2.0,Terminated,5521030.0,5526746.0,600.0,20.000000,NaN,NaN,4ac7f9c48a6913dfa81169f7d6a150fa,bert,a5d7f3cf14cc3baef1b59eab27895e2222f2302735d873...,c34f6c58302ec92032293a469d2cbfaccba65ff97203b6...,d4f54f7e5cf18341942c47ef,/dev/nvidia0,110.590788,13.005300,2.357810,3.056641,0.645605,0.827148,1.037695e+06,1.527555e+05,201.449250,181.084731,5716.0,P100,64.0,512.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e49cae3b33b5c205ed8542b0,5283a27d4f19d767256f97c18880a70d69f4d97ca655a3...,47baba33cffe,ps,1.0,Terminated,5890601.0,5893675.0,600.0,29.296875,NaN,NaN,7389796d26bc0d9c8bae9700a46e88ca,xlnet,8d6b9f3b9ef8539217bc16388fb88e18542c0048677344...,20fb53a63dab2ac5da0e2bc08328ee3835d5b0bf0a78f2...,2988f6d00a08192b50b767f4,/dev/nvidia1,127.340568,81.609635,1.940527,1.995117,1.200031,1.213867,1.941028e+06,1.165604e+06

## Simple stats

Top-3 GPUs for the profile 8 (the largest cluster)

In [22]:
gpu_type_machine_spec = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.profile == 8]['gpu_type_machine_spec'].values

gpu_type_stats = Counter(gpu_type_machine_spec)

gpu_type_stats.most_common(3)


[('P100', 7644), ('MISC', 3636), ('T4', 864)]